In [1]:
import pandas as pd
import numpy as np
from scipy import optimize

import os

def iones():
    nombre = 'iones'
    path = os.getcwd() + '/templates/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    ion_df = pd.read_csv(dict_csv[nombre]).fillna(0)

    indice = ion_df['GreenHow']

    copy = ion_df.copy()
    del copy['GreenHow']
    del copy['ingredientes']

    A = copy.to_numpy()
    AT = A.transpose()
    
    return ion_df, indice, A, AT


def objetivo(nombre):
    path = os.getcwd() + '/inputs/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    df = pd.read_csv(dict_csv[nombre]).fillna(0)

    target = df.iloc[0].to_numpy()[2:]
    const = df['VALOR'].to_numpy()[1:]

    return target, const

def corrida(indice, constraints, matriz):

    lista = []
    index = []

    for i in range(len(indice)):
        if constraints[i] == 1:
            print('Añadiendo: ' + indice[i])
            lista.append(matriz[i])
            index.append(indice[i])
        if constraints[i] == 0:
            print('No se utilizara : ' + indice[i])
    A = np.array(lista)

    AT = A.transpose()

    return A, AT, index

def loss(x0, k, A):
    lista = [sum((k - np.dot(x0, A))**2)/k.shape[0] for i in range(x0.shape[0])]
    
    return sum(lista)

def estefania(guardar=False):

    variedad = input('Ingrese el nombre de la variedad: ')

    df, indice, matriz, trans = iones()

    elementos = list(df.iloc[0][2:].index)

    target, const = objetivo(variedad)

    A, AT, nuevoindice = corrida(indice, const, matriz)

    x = np.random.rand(A.shape[0]) 

    print('X = ' , x)

    loss(x, target, A)

    res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
                        method='L-BFGS-B', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='nelder-mead', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='SLSQP', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='BFGS', bounds=[(0,None)]*x.shape[0])    


    # resultados
    d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

    # % de error

    error = (target - np.dot(res.x, A))*100/1

    err = [round(num, 2) for num in error]

    e = pd.DataFrame({'error[%]': err})

    resultados = pd.DataFrame(elementos, columns=['elemento'])
    resultados = resultados.join(e)

    resultados = d.join(resultados)

    if guardar == True:

        current = os.getcwd()
        # guardar resultados
        resultados.to_csv(current + '/outputs/' + 'resultados' + variedad + '.csv', index=False)


    return resultados


</b>
Electronic<br>
Salt <br>
Target <br>
Estimator <br>
For <br>
Advanced <br>
Nutrition <br>
In <br>
Agriculture <br>

# ESTEFANIA

In [3]:
estefania(guardar=True)

No se utilizara : Ácido nítrico
No se utilizara : Ácido fosfórico
No se utilizara : Ácido sulfúrico
Añadiendo: Nitrato de amonio, Nitamks
Añadiendo: Nitrato de Potasio más Mg, KERF
Añadiendo: Nitrato de potasio más S, Nitroksul
Añadiendo: Nitrato de calcio samurai
Añadiendo: Nitrato de calcio bi
Añadiendo: Nitrato de magnesio
Añadiendo: Fosfato monoamónico, Map colossal
Añadiendo: Fosfato monopotásico, Mkp colossal
Añadiendo: Sulfato de amonio
Añadiendo: Sulfato de potasio, Solucros
Añadiendo: Sulfato de calcio
Añadiendo: Sulfato de magnesio, Sulmag
No se utilizara : Sulfato de fierro
No se utilizara : Sulfato de manganeso
No se utilizara : Sulfato de zinc
No se utilizara : Sulfato de cobre
Añadiendo: Quelato Ca 95%
Añadiendo: Quelato Fe 6%, microhow
Añadiendo: Quelato de Mn 13%, microhow
Añadiendo: Quelato de Zn 15%, microhow
Añadiendo: Quelato Cu 15%, microhow
No se utilizara : Quelato full mix
No se utilizara : Quelato full mix B
Añadiendo: Ácido bórico, abo
No se utilizara : Solubo

/var/folders/gw/28wn0d255f9grd5k4bcn_s0m0000gn/T/ipykernel_1425/2354595970.py:76: OptimizeWarning: Unknown solver options: tol
  res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},



At iterate   49    f=  1.57458D-02    |proj g|=  3.33517D-01
  Positive dir derivative in projection 
  Using the backtracking step 

At iterate   50    f=  1.54115D-02    |proj g|=  3.62477D-01

At iterate   51    f=  1.39484D-02    |proj g|=  1.99816D-01

At iterate   52    f=  1.38459D-02    |proj g|=  1.39231D-01

At iterate   53    f=  1.38233D-02    |proj g|=  1.39519D-01
  Positive dir derivative in projection 
  Using the backtracking step 

At iterate   54    f=  1.34058D-02    |proj g|=  3.30086D-01

At iterate   55    f=  1.32517D-02    |proj g|=  2.42216D-01

At iterate   56    f=  1.29677D-02    |proj g|=  8.74303D-02

At iterate   57    f=  1.28493D-02    |proj g|=  1.15051D-01

At iterate   58    f=  1.27507D-02    |proj g|=  1.75860D-01

At iterate   59    f=  1.26446D-02    |proj g|=  2.29217D-01

At iterate   60    f=  1.24333D-02    |proj g|=  2.67313D-01

At iterate   61    f=  1.19878D-02    |proj g|=  3.26882D-01

At iterate   62    f=  1.10609D-02    |proj g|=  

,fertilizante,g/l,elemento,error[%]
0,"Nitrato de amonio, Nitamks",0.000077,H+,0.00
1,"Nitrato de Potasio más Mg, KERF",0.003547,NO3-,-0.02
2,"Nitrato de potasio más S, Nitroksul",0.833771,NH4+,-0.20
3,Nitrato de calcio samurai,0.615280,H2PO4-,0.34
4,Nitrato de calcio bi,0.423137,K+,-0.01
5,Nitrato de magnesio,0.257141,Ca+2,-0.29
6,"Fosfato monoamónico, Map colossal",0.001129,Mg+2,-0.28
7,"Fosfato monopotásico, Mkp colossal",0.273974,SO4-2,-0.22
8,Sulfato de amonio,0.000000,Fe,-0.34
9,"Sulfato de potasio, Solucros",0.098514,Mn,0.15


### paso a paso: 

## Cargamos los datos de las sales

In [19]:
df, indice, matriz, trans = iones()

print('Las sales disponibles para el estudio son: ' , indice)

elementos = list(df.iloc[0][2:].index)


Las sales disponibles para el estudio son:  0                           Ácido nítrico
1                         Ácido fosfórico
2                         Ácido sulfúrico
3              Nitrato de amonio, Nitamks
4         Nitrato de Potasio más Mg, KERF
5     Nitrato de potasio más S, Nitroksul
6               Nitrato de calcio samurai
7                    Nitrato de calcio bi
8                     Nitrato de magnesio
9       Fosfato monoamónico, Map colossal
10     Fosfato monopotásico, Mkp colossal
11                      Sulfato de amonio
12           Sulfato de potasio, Solucros
13                      Sulfato de calcio
14            Sulfato de magnesio, Sulmag
15                      Sulfato de fierro
16                   Sulfato de manganeso
17                        Sulfato de zinc
18                       Sulfato de cobre
19                         Quelato Ca 95%
20                Quelato Fe 6%, microhow
21            Quelato de Mn 13%, microhow
22            Quelato de Zn 15%,

## Cargamos la solución objetivo

In [20]:
variedad = 'kale-001'

target, const = objetivo(variedad)


for i in range(len(elementos)):
    print('El objetivo del ' + elementos[i] + ' es: ' +  str(target[i]) + ' g/l')

El objetivo del H+ es: 0.0 g/l
El objetivo del NO3- es: 9.38693625193657 g/l
El objetivo del NH4+ es: 4.17800052831859 g/l
El objetivo del H2PO4- es: 1.12998866589083 g/l
El objetivo del K+ es: 5.37221795855717 g/l
El objetivo del Ca+2 es: 3.74270173162333 g/l
El objetivo del Mg+2 es: 1.85147089076322 g/l
El objetivo del SO4-2 es: 2.1830656479027 g/l
El objetivo del Fe es: 0.01790670606142 g/l
El objetivo del Mn es: 0.0091008372770294 g/l
El objetivo del Zn es: 0.0023604947597016 g/l
El objetivo del Cu es: 0.0015736631731344 g/l
El objetivo del B es: 0.0462534690101758 g/l
El objetivo del Mo es: 0.0005211590577444 g/l
El objetivo del Co es: 0.0 g/l
El objetivo del Cl- es: 0.0 g/l
El objetivo del Na+ es: 0.0 g/l


## Hacemos la corrida con los fertilizantes seleccionados

In [21]:
A, AT, nuevoindice = corrida(indice, const, matriz)

No se utilizara : Ácido nítrico
No se utilizara : Ácido fosfórico
No se utilizara : Ácido sulfúrico
Añadiendo: Nitrato de amonio, Nitamks
Añadiendo: Nitrato de Potasio más Mg, KERF
Añadiendo: Nitrato de potasio más S, Nitroksul
Añadiendo: Nitrato de calcio samurai
Añadiendo: Nitrato de calcio bi
Añadiendo: Nitrato de magnesio
Añadiendo: Fosfato monoamónico, Map colossal
Añadiendo: Fosfato monopotásico, Mkp colossal
Añadiendo: Sulfato de amonio
Añadiendo: Sulfato de potasio, Solucros
Añadiendo: Sulfato de calcio
Añadiendo: Sulfato de magnesio, Sulmag
No se utilizara : Sulfato de fierro
No se utilizara : Sulfato de manganeso
No se utilizara : Sulfato de zinc
No se utilizara : Sulfato de cobre
No se utilizara : Quelato Ca 95%
Añadiendo: Quelato Fe 6%, microhow
Añadiendo: Quelato de Mn 13%, microhow
Añadiendo: Quelato de Zn 15%, microhow
Añadiendo: Quelato Cu 15%, microhow
No se utilizara : Quelato full mix
No se utilizara : Quelato full mix B
Añadiendo: Ácido bórico, abo
No se utilizara :

In [22]:
for i in range(len(nuevoindice)):
    print('El ' + nuevoindice[i] + ' es el # ' + str(i)  )


El Nitrato de amonio, Nitamks es el # 0
El Nitrato de Potasio más Mg, KERF es el # 1
El Nitrato de potasio más S, Nitroksul es el # 2
El Nitrato de calcio samurai es el # 3
El Nitrato de calcio bi es el # 4
El Nitrato de magnesio es el # 5
El Fosfato monoamónico, Map colossal es el # 6
El Fosfato monopotásico, Mkp colossal es el # 7
El Sulfato de amonio es el # 8
El Sulfato de potasio, Solucros es el # 9
El Sulfato de calcio es el # 10
El Sulfato de magnesio, Sulmag es el # 11
El Quelato Fe 6%, microhow es el # 12
El Quelato de Mn 13%, microhow es el # 13
El Quelato de Zn 15%, microhow es el # 14
El Quelato Cu 15%, microhow es el # 15
El Ácido bórico, abo es el # 16
El Molibdato de amonio es el # 17


## Creamos el vector a optimizar 

In [33]:
x = np.random.rand(A.shape[0])
print('Vector aleatoreo de longitud ' + str(len(x)) + ':', x)

Vector aleatoreo de longitud 18: [0.11370259 0.00186819 0.05398221 0.21239596 0.79472313 0.47896618
 0.02006729 0.60191672 0.12749124 0.45687474 0.73716968 0.34438887
 0.67711501 0.75684573 0.40485104 0.79263727 0.53209124 0.60728082]


In [34]:
x[17] = 0.00013

x


array([1.13702594e-01, 1.86818504e-03, 5.39822133e-02, 2.12395960e-01,
       7.94723127e-01, 4.78966184e-01, 2.00672908e-02, 6.01916718e-01,
       1.27491237e-01, 4.56874735e-01, 7.37169681e-01, 3.44388869e-01,
       6.77115011e-01, 7.56845734e-01, 4.04851039e-01, 7.92637274e-01,
       5.32091241e-01, 1.30000000e-04])

# Función perdida 

In [35]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('target es de dimension: ', target.shape)
print('x es de dimension: ', x.shape)


A es de dimension:  (18, 17)
AT es de dimension:  (17, 18)
target es de dimension:  (17,)
x es de dimension:  (18,)


In [36]:
loss(x, target, A)

216.91993697890973

In [37]:
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='SLSQP')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='BFGS')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='Nelder-Mead')
res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True}, 
                        method='L-BFGS-B', bounds=[(0,1)]*x.shape[0])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           18     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.16920D+02    |proj g|=  9.99870D-01

At iterate    1    f=  1.39871D+02    |proj g|=  6.19964D-01

At iterate    2    f=  3.47112D+01    |proj g|=  1.00000D+00

At iterate    3    f=  1.53151D+01    |proj g|=  1.00000D+00

At iterate    4    f=  4.20237D+00    |proj g|=  1.00000D+00

At iterate    5    f=  2.76512D+00    |proj g|=  1.00000D+00

At iterate    6    f=  2.40283D+00    |proj g|=  1.00000D+00

At iterate    7    f=  2.10220D+00    |proj g|=  1.00000D+00

At iterate    8    f=  1.57707D+00    |proj g|=  1.00000D+00

At iterate    9    f=  1.21636D+00    |proj g|=  1.00000D+00

At iterate   10    f=  9.39948D-01    |proj g|=  9.99769D-01

At iterate   11    f=  7.25015D-01    |proj g|=  7.87990D-01

At iterate   12    f=  7.12502D-01    |proj g|=  9.04072D-01

At iterate   13    f=  6.9

In [38]:
# resultados
d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

# % de error

error = (target - np.dot(res.x, A))*100/1

err = [round(num, 2) for num in error]

e = pd.DataFrame({'error[%]': err})

resultados = pd.DataFrame(elementos, columns=['elemento'])
resultados = resultados.join(e)

resultados = d.join(resultados)
resultados

,fertilizante,g/l,elemento,error[%]
0,"Nitrato de amonio, Nitamks",0.000000,H+,0.00
1,"Nitrato de Potasio más Mg, KERF",0.439304,NO3-,-11.29
2,"Nitrato de potasio más S, Nitroksul",0.000000,NH4+,8.76
3,Nitrato de calcio samurai,0.000000,H2PO4-,-8.73
4,Nitrato de calcio bi,0.644138,K+,8.86
5,Nitrato de magnesio,0.000000,Ca+2,22.91
6,"Fosfato monoamónico, Map colossal",0.110034,Mg+2,22.61
7,"Fosfato monopotásico, Mkp colossal",0.038745,SO4-2,-22.40
8,Sulfato de amonio,0.000000,Fe,0.00
9,"Sulfato de potasio, Solucros",0.000000,Mn,-0.01


## Gradiente descendente

In [410]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('k es de dimension: ', k.shape)
print('x es de dimension: ', x.shape)

A es de dimension:  (14, 7)
AT es de dimension:  (7, 14)
k es de dimension:  (7,)
x es de dimension:  (14,)


In [69]:
grad = lambda x: (-2/7)*(sum([np.dot((k[i] - np.dot(x, A[:,i])), A[:,i]) for i in range(A.shape[1])]))  

x = np.random.rand(A.shape[0])


print(x)
grad(x[1])


[0.48493831 0.82158739 0.92309745 0.32309576 0.85806003 0.52072956
 0.79377436 0.42070142 0.87617722 0.79958896 0.51304218 0.68733127
 0.16162787 0.34753273]


110.85150861358929

In [90]:
itermax = 5
iternum = 1
alfa = 0.01

x = np.random.rand(A.shape[0])


while True:

    xnuevo = np.ones(x.shape[0])
    print(xnuevo)
    for i in range(x.shape[0]):
        xnuevo[i] *=  x[i] - alfa*grad(x[i])
    print(xnuevo)
    x = xnuevo.copy()

    iternum += 1
    print('Iteracion #' + str(iternum))

    if iternum == itermax:
        print(xnuevo)
        break

    



[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 0.38803855  2.40041526  2.88434117  2.93456591 -0.98157776  2.78440451
 -0.57346545 -0.12360223 -0.18617435  1.15120954 -0.93201978 -0.23402997
 -0.79617713  2.45360293]
Iteracion #2
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.44611281 -6.59803364 -8.53244826 -8.73321343  6.92092982 -8.13296781
  5.28956766  3.49131309  3.74143488 -1.60453827  6.7228299   3.93272988
  6.17982115 -6.81064265]
Iteracion #3
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ -2.78336583  29.37179276  37.10429853  37.90682442 -24.66805056
  35.50744083 -18.14694738 -10.95871912 -11.95854003   9.41111246
 -23.87617857 -12.72321049 -21.70558996  30.22166246]
Iteracion #4
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[  14.12328266 -114.41169989 -145.32112585 -148.5290917   101.60372728
 -138.93794862   75.53668483   46.80291912   50.79953953  -34.62214802
   98.4383486    53.85618451   89.76177599 -117.80891489]
Iteracion #5
[  14.12328266 -114.41169989 -145.32112585

In [84]:
error = k - np.dot(x1, A)
for item in error:
    print(str(((item*100)/1).round(4)) + ' % de error para el elemento # ' + str(1 + error.tolist().index(item))) 

-2329.0681 % de error para el elemento # 1
-1184.4613 % de error para el elemento # 2
-832.3824 % de error para el elemento # 3
-1829.7247 % de error para el elemento # 4
-1094.4554 % de error para el elemento # 5
-298.0074 % de error para el elemento # 6
-1737.585 % de error para el elemento # 7
